In [1]:
# Import required libraries
import os
from azure.identity import DefaultAzureCredential
from azure.identity import AzureCliCredential
from azure.ai.ml import automl, Input, MLClient, command

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data
from azure.ai.ml.automl import (
    classification,
    ClassificationPrimaryMetrics,
    ClassificationModels,
)

## Initialize MLClient with Azure Default Credentials

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
except Exception as ex:
    print(ex)

In [3]:

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [4]:

my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:bankdata:1"
)

In [5]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="minecompute",
    experiment_name="bank-class-dev",
    training_data=my_training_data_input,
    target_column_name="y",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)
     

In [6]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/loving_orange_bhp569f2d4?wsid=/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourcegroups/myminerg/workspaces/mineworkspace&tid=215b7ce2-5263-4593-a622-da030405d151


# Retrieve the Best Trial (Best Model's trial/run)

In [7]:
import mlflow

In [8]:
# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourceGroups/mymineRG/providers/Microsoft.MachineLearningServices/workspaces/mineworkspace


In [9]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourceGroups/mymineRG/providers/Microsoft.MachineLearningServices/workspaces/mineworkspace


In [10]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [11]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.rootRunId': 'loving_orange_bhp569f2d4',
 'mlflow.runName': 'loving_orange_bhp569f2d4',
 'mlflow.user': 'Monsurat Ayinde',
 'model_explain_run': 'best_run'}>, info=<RunInfo: artifact_uri='azureml://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourceGroups/mymineRG/providers/Microsoft.MachineLearningServices/workspaces/mineworkspace/experiments/9034bfb4-6f87-4422-b6fb-3f250df7fc54/runs/loving_orange_bhp569f2d4/artifacts', end_time=None, experiment_id='9034bfb4-6f87-4422-b6fb-3f250df7fc54', lifecycle_stage='active', run_id='loving_orange_bhp569f2d4', run_name='loving_orange_bhp569f2d4', run_uuid='loving_orange_bhp569f2d4', start_time=0, status='SCHEDULED', user_id='802f6321-c4ce-4191-8947-55f1ce572875'>, inputs=<RunInputs: dataset_inputs=[]>>


In [12]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'model_explain_run': 'best_run', 'mlflow.rootRunId': 'loving_orange_bhp569f2d4', 'mlflow.runName': 'loving_orange_bhp569f2d4', 'mlflow.user': 'Monsurat Ayinde'}


In [13]:
print("Available tags: ", mlflow_parent_run.data.tags)


Available tags:  {'model_explain_run': 'best_run', 'mlflow.rootRunId': 'loving_orange_bhp569f2d4', 'mlflow.runName': 'loving_orange_bhp569f2d4', 'mlflow.user': 'Monsurat Ayinde'}


In [14]:


# Attempt to get the best model's child run using 'automl_best_child_run_id' tag
best_child_run_id = mlflow_parent_run.data.tags.get("automl_best_child_run_id")

if best_child_run_id:
    print("Found best child run id: ", best_child_run_id)
    best_run = mlflow_client.get_run(best_child_run_id)
else:
    print("'automl_best_child_run_id' tag not found. Searching for best child run manually...")
    
    # Search for child runs of the parent run
    parent_run_id = mlflow_parent_run.info.run_id
    child_runs = mlflow.search_runs([parent_run_id])
    
    if len(child_runs) > 0:
        # Sort child runs based on a metric, e.g., accuracy or any performance measure you care about
        best_run = child_runs.iloc[child_runs['metrics.accuracy'].idxmax()]
        best_child_run_id = best_run.run_id
        print(f"Found best child run id: {best_child_run_id} with accuracy: {best_run['metrics.accuracy']}")
    else:
        print("No child runs found for the given parent run.")
    
# If a best run is found, you can proceed to work with it
if best_child_run_id:
    best_run = mlflow_client.get_run(best_child_run_id)
    print("Best child run details: ")
    print(best_run)
else:
    print("Unable to find a best child run.")


'automl_best_child_run_id' tag not found. Searching for best child run manually...
No child runs found for the given parent run.
Unable to find a best child run.


In [15]:
# Check the status of the parent run
print("Parent run status: ", mlflow_parent_run.info.status)


Parent run status:  SCHEDULED


In [20]:
# Force refresh the parent run info
mlflow_parent_run = mlflow_client.get_run(mlflow_parent_run.info.run_id)
print("Updated Parent run status: ", mlflow_parent_run.info.status)


Updated Parent run status:  FINISHED


In [21]:
print("Parent run details:")
print(mlflow_parent_run)


Parent run details:
<Run: data=<RunData: metrics={'AUC_macro': 0.9441532035778856,
 'AUC_micro': 0.9794639565349771,
 'AUC_weighted': 0.9441532035778856,
 'accuracy': 0.9154303550591765,
 'average_precision_score_macro': 0.824234646373134,
 'average_precision_score_micro': 0.9799935948713918,
 'average_precision_score_weighted': 0.9544501860116796,
 'balanced_accuracy': 0.7644699858498146,
 'f1_score_macro': 0.7776272076839497,
 'f1_score_micro': 0.9154303550591765,
 'f1_score_weighted': 0.9132663402966031,
 'log_loss': 0.2397423886037192,
 'matthews_correlation': 0.5567581616167823,
 'norm_macro_recall': 0.5289399716996291,
 'precision_score_macro': 0.7931017399250836,
 'precision_score_micro': 0.9154303550591765,
 'precision_score_weighted': 0.9117058604455106,
 'recall_score_macro': 0.7644699858498146,
 'recall_score_micro': 0.9154303550591765,
 'recall_score_weighted': 0.9154303550591765,
 'weighted_accuracy': 0.9531328359988777}, params={}, tags={'automl_best_child_run_id': 'lovin

In [22]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  loving_orange_bhp569f2d4_3
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9441532035778856,
 'AUC_micro': 0.9794639565349771,
 'AUC_weighted': 0.9441532035778856,
 'accuracy': 0.9154303550591765,
 'average_precision_score_macro': 0.824234646373134,
 'average_precision_score_micro': 0.9799935948713918,
 'average_precision_score_weighted': 0.9544501860116796,
 'balanced_accuracy': 0.7644699858498146,
 'f1_score_macro': 0.7776272076839497,
 'f1_score_micro': 0.9154303550591765,
 'f1_score_weighted': 0.9132663402966031,
 'log_loss': 0.2397423886037192,
 'matthews_correlation': 0.5567581616167823,
 'norm_macro_recall': 0.5289399716996291,
 'precision_score_macro': 0.7931017399250836,
 'precision_score_micro': 0.9154303550591765,
 'precision_score_weighted': 0.9117058604455106,
 'recall_score_macro': 0.7644699858498146,
 'recall_score_micro': 0.9154303550591765,
 'recall_score_weighted': 0.9154303550591765,
 'weighted_accuracy': 0.9531328359988777}, p

In [23]:
best_run.data.metrics

{'average_precision_score_weighted': 0.9544501860116796,
 'precision_score_weighted': 0.9117058604455106,
 'precision_score_micro': 0.9154303550591765,
 'recall_score_macro': 0.7644699858498146,
 'precision_score_macro': 0.7931017399250836,
 'norm_macro_recall': 0.5289399716996291,
 'average_precision_score_macro': 0.824234646373134,
 'accuracy': 0.9154303550591765,
 'weighted_accuracy': 0.9531328359988777,
 'recall_score_weighted': 0.9154303550591765,
 'AUC_weighted': 0.9441532035778856,
 'log_loss': 0.2397423886037192,
 'recall_score_micro': 0.9154303550591765,
 'f1_score_macro': 0.7776272076839497,
 'average_precision_score_micro': 0.9799935948713918,
 'f1_score_micro': 0.9154303550591765,
 'balanced_accuracy': 0.7644699858498146,
 'AUC_micro': 0.9794639565349771,
 'f1_score_weighted': 0.9132663402966031,
 'matthews_correlation': 0.5567581616167823,
 'AUC_macro': 0.9441532035778856}

#  Register Best Model and Deploy

##  Create managed online endpoint

In [24]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [34]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "bankmarketing2-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
)

In [35]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://bankmarketing2-09272004550748.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://bankmarketing2-09272004550748.eastus.inference.ml.azure.com/swagger.json', 'name': 'bankmarketing2-09272004550748', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {}, 'properties': {'createdBy': 'Monsurat Ayinde', 'createdAt': '2024-09-27T20:05:07.169896+0000', 'lastModifiedAt': '2024-09-27T20:05:07.169896+0000', 'azureml.onlineendpointid': '/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourcegroups/myminerg/providers/microsoft.machinelearningservices/workspaces/mineworkspace/onlineendpoints/bankmarketing2-09272004550748', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:25334aea-01d0-445e-867c-a2114a

# Register best model and deploy

## Register model

In [36]:
model_name = "bankmarketing-model2"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample classification model",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)

registered_model = ml_client.models.create_or_update(model)

In [37]:
registered_model.id

'/subscriptions/952fe447-cc92-4a68-a6bd-52160cc69c10/resourceGroups/myminerg/providers/Microsoft.MachineLearningServices/workspaces/mineworkspace/models/bankmarketing-model2/versions/1'

## Deploy

In [39]:
deployment = ManagedOnlineDeployment(
    name="bankmarketing-deploy2",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [40]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint bankmarketing2-09272004550748 exists


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {"error":{"code":"Validation","message":"{\"errors\":{\"VmSize\":[\"Not enough quota available for Standard_DS3_v2 in SubscriptionId 952fe447-cc92-4a68-a6bd-52160cc69c10. Current usage/limit: 6/6. Additional needed: 8 Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-6b2a65f98b768e168e36de82d1ebca3d-6832b58c8239f1ff-01\"}"}}
	Code: InferencingClientCallFailed
	Message: {"error":{"code":"Validation","message":"{\"errors\":{\"VmSize\":[\"Not enough quota available for Standard_DS3_v2 in SubscriptionId 952fe447-cc92-4a68-a6bd-52160cc69c10. Current usage/limit: 6/6. Additional needed: 8 Please see troubleshooting guide, available here: https://aka.

In [41]:
# bankmarketing deployment to take 100% traffic
endpoint.traffic = {"bankmarketing-deploy": 100}
ml_client.begin_create_or_update(endpoint)

HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {"error":{"code":"Validation","message":"{\"errors\":{\"DeploymentWeights\":[\"Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [bankmarketing-deploy]\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00-39ca5b5f76864f8bcb5e3db1e928de23-a05635625865da80-01\"}"}}
	Code: InferencingClientCallFailed
	Message: {"error":{"code":"Validation","message":"{\"errors\":{\"DeploymentWeights\":[\"Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [bankmarketing-deploy]\"]},\"type\":\"https://tools.ietf.org/html/rfc9110#section-15.5.1\",\"title\":\"One or more validation errors occurred.\",\"status\":400,\"traceId\":\"00